In [ ]:
import os, glob, cv2, math, csv, tqdm
import numpy as np
from PIL import Image
import util 
from keras.utils.training_utils import multi_gpu_model
from keras import backend as k
from keras import applications, optimizers, losses, layers
from keras.utils.training_utils import multi_gpu_model
from keras import layers ,losses
from keras.layers import Dropout, Input, Flatten, Dense, GlobalAveragePooling2D, Activation, LSTM, SeparableConv2D, TimeDistributed, GRU
from keras.models import Sequential, Model, load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.convolutional import Conv2D
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def test_generator(root, seq, n_frame, batch):
    seq_pose = seq +'.txt'
    img_dir = os.path.join(root, 'sequences', seq,'image_2')
    lab_dir = os.path.join(root, 'poses', seq_pose)
    img_list = sorted(os.listdir(img_dir))
    lab_list = util.load_dof6_labels(lab_dir)
    idx = n_frame
    while 1:
        img = []
        lab = []
        
        for i in range(batch):
            bat_img = []
            bat_lab = []
            #print(idx)
            img_seq_list = img_list[idx:idx+n_frame+1]
            bat_lab = lab_list[idx+n_frame]
            for img_name in img_seq_list:
                img_path = os.path.join(img_dir, img_name)
                tmp_img = cv2.resize(np.array(Image.open(img_path)),(512,384))/255.
                bat_img.append(tmp_img)
            img.append(np.concatenate((np.array(bat_img)[1:], np.array(bat_img)[:-1]),axis = 3))
            lab.append(np.array(bat_lab))
        print(idx)
        idx = idx + 1
        yield np.array(img), [np.array(lab)[:,:,:3],np.array(lab)[:,:,3:]]

In [ ]:
def data_generator(root, seq, n_frame, batch):
    while 1:
        seq_ch = np.random.choice(seq)
        seq_pose = seq_ch +'.txt'
        img_dir = os.path.join(root, 'sequences', seq_ch,'image_2')
        lab_dir = os.path.join(root, 'poses', seq_pose)
        img_list = sorted(os.listdir(img_dir))
        lab_list = util.load_dof6_labels(lab_dir)
        img = []
        lab = []
        for i in range(batch):
            bat_img = []
            bat_lab = []
            idx = np.random.randint(0, len(img_list)-n_frame+1)
            img_seq_list = img_list[idx:idx+n_frame+1]
            bat_lab = lab_list[idx:idx+n_frame]
            for img_name in img_seq_list:
                img_path = os.path.join(img_dir, img_name)
                tmp_img = cv2.resize(np.array(Image.open(img_path)),(512,384))/255.
                bat_img.append(tmp_img)
            img.append(np.concatenate((np.array(bat_img)[1:], np.array(bat_img)[:-1]),axis = 3))
            lab.append(np.array(bat_lab))
        yield np.array(img), [np.array(lab)[:,:,:3],np.array(lab)[:,:,3:]]

In [ ]:
n_time = 10
batch = 4

train_seq = ['00','01','02','03','04','05','06','07','08']
train_root= '/data1/Kitti/odometry/'
train_generator = data_generator(train_root, train_seq, n_time, batch)

# val_seq = ['08','09']
# val_root= '/data1/Kitti/odometry/'
# val_generator = data_generator(val_root, val_seq, n_time, batch)

In [ ]:
model_gen = load_model('../GAN/flow_net_good_e5.h5')
model_top = Model(inputs = model_gen.input, 
                  outputs=model_gen.layers[18].output)

In [ ]:
model_top.summary()

In [ ]:
inputs = Input(shape=(None, 384, 512,6))
x = TimeDistributed(model_top)(inputs)
x = TimeDistributed(Flatten())(x)

x = LSTM(1024, return_sequences=True)(x)
x = LSTM(1024, return_sequences=True)(x)
rotation = Dense(512, activation='relu')(x)
rotation = Dense(3, name = 'rotation')(rotation)
position = Dense(512, activation='relu')(x)
position = Dense(3, name = 'position')(position)

In [ ]:
model = Model(inputs = inputs, outputs = [rotation, position])
for i in range(2):
    model.layers[i].trainable = False
model.compile(optimizer='adam', loss=['mse','mse'],loss_weights=[100,1])
model.summary()

In [ ]:
hist = model.fit_generator(train_generator, steps_per_epoch=1000, epochs=5)    

In [ ]:
#plt.plot(hist.history['loss'],'r-')
plt.plot(hist.history['rotation_loss'],'b-')
plt.plot(hist.history['position_loss'],'g-')
plt.show()

In [ ]:
test_seq = '10'
test_root= './'
test_n_time = 10
test_batch = 1
test_generator = test_generator(test_root, test_seq, test_n_time, test_batch)

In [ ]:
test_img, test_lab = next(test_generator)
test_img.shape
np.array(test_lab).shape

In [ ]:
predic = []
labels = []
for i in range(1000):
    test_img, test_lab = next(test_generator)
    labels.append(np.reshape(np.concatenate((np.array(test_lab)[0],np.array(test_lab)[1]),axis=2),(10,6))[-1,:])
    predic_tmp = model.predict(test_img)
    predic.append(np.reshape(np.concatenate((np.array(predic_tmp)[0],np.array(predic_tmp)[1]),axis=2),(10,6))[-1,:])

In [ ]:
predic_np = np.array(predic)
labels_np = np.array(labels)

In [ ]:
predic_np.shape

In [ ]:
path = util.dof6_to_eval_path(predic_np)
path_gt = util.dof6_to_eval_path(labels_np)

plt.plot(path[0,:],path[2,:],'r')
plt.plot(path_gt[0,:],path_gt[2,:],'g')
plt.show()

In [ ]:
plt.plot(path[1,:],'r')
plt.plot(path_gt[1,:],'g')
plt.show()

In [ ]:
plt.plot(path[0,:],'r')
plt.plot(path_gt[0,:],'g')
plt.show()

In [ ]:
plt.plot(path[2,:],'r')
plt.plot(path_gt[2,:],'g')
plt.show()

In [ ]:
test_img, test_lab = next(train_generator)
pre = model_gen.predict(test_img[0])
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.imshow(test_img[0][0][:,:,:3])
plt.title("input(val)")
plt.subplot(122)
plt.imshow(test_img[0][0][:,:,3:])
plt.show()
            
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.imshow(pre[3][0])
plt.show()
test_img.shape

print(np.array(test_img).shape,np.array(test_lab).shape)



In [ ]:
model.save('rcnn_odom_6ep_best2.h5')